In [1]:
import pandas as pd
import numpy as np
import csv
from pandasql import sqldf

In [4]:
customer_list = pd.read_table('user_list.txt',sep='\t')  
print(customer_list)


                                                   org_id  remap_id
0       be1981ab818cf4ef6765b2ecaea7a2cbf14ccd6e8a7ee9...         0
1       b4db5e5259234574edfff958e170fe3a5e13b6f146752c...         1
2       49beaacac0c7801c2ce2d189efe525fe80b5d37e46ed05...         2
3       a65f77281a528bf5c1e9f270141d601d116e1df33bf9df...         3
4       cd04ec2726dd58a8c753e0d6423e57716fd9ebcf2f14ed...         4
...                                                   ...       ...
676039  7d8e464f86e00f7c7d03d84ef10aa00c1cdda2f02d4679...    676039
676040  2d2eb3bc988bfdb60bd8e0b0eff29755e52b06a47f8168...    676040
676041  350278b8dae43e5f61df3541e77d765beae2ae4125bdf2...    676041
676042  0906d87e968fb6f872ece86ac02d4cc6e503d71b9df052...    676042
676043  9d05cccf0c8481b445b322a179c5471c10a3833adf06a2...    676043

[676044 rows x 2 columns]


In [3]:
transaction = pd.read_csv("transactions_train.csv", header=0)
last_three_months_tran = transaction[transaction["t_dat"].str.match(pat = '(((2020-07)|(2020-08)|(2020-09)).)')]
last_three_months_tran

,t_dat,customer_id,article_id,price,sales_channel_id
28401361,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,825577003,0.013542,2
28401362,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,849487005,0.042356,2
28401363,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,772659001,0.023712,2
28401364,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0.042356,2
28401365,2020-07-01,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004,0.042356,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [5]:
new_transaction = pd.DataFrame(last_three_months_tran, columns=["customer_id", "article_id"])
new_transaction

,customer_id,article_id
28401361,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,825577003
28401362,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,849487005
28401363,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,772659001
28401364,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004
28401365,0001d44dbe7f6c4b35200abdb052c77a87596fe1bdcc37...,804732004
...,...,...
31788319,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001
31788320,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004
31788321,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001
31788322,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002


In [6]:
customer_purchase_detail = sqldf("""SELECT remap_id as customer, article_id 
        FROM customer_list as c
        INNER JOIN new_transaction as t
        on c.org_id = t.customer_id
        group by remap_id, article_id
    """)
print(customer_purchase_detail)

         customer  article_id
0               0   186262001
1               0   214844001
2               0   253448001
3               0   253448003
4               0   253448059
...           ...         ...
2693343    676029   872707005
2693344    676031   816166008
2693345    676043   757828001
2693346    676043   759871025
2693347    676043   892408008

[2693348 rows x 2 columns]


In [7]:
item_list = pd.read_table('item_list.txt',sep='\t')  
# item_list
customer_purchase_remapid = sqldf("""SELECT customer, remap_id 
        FROM customer_purchase_detail as c
        INNER JOIN item_list as i
        on c.article_id = i.org_id
    """)
print(customer_purchase_remapid)

         customer  remap_id
0               0       137
1               0       285
2               0       504
3               0       506
4               0       508
...           ...       ...
2693343    676029     97058
2693344    676031     84530
2693345    676043     69638
2693346    676043     70234
2693347    676043    100783

[2693348 rows x 2 columns]


In [8]:
f = open('total.txt','a')
for i in range(676044): # 676044 is the total number of the customers
    cur_line = customer_purchase_remapid[customer_purchase_remapid['customer'] == i]
    cur_line_new = np.asarray(cur_line['remap_id'])
    str_cur_line = " ".join([str(x) for x in cur_line_new])
    str_cur_line = str(i) + " " + str_cur_line
    f.write(str_cur_line)
    f.write("\n")
 
f.close()


In [9]:
f1 = open('train_1.txt','a')
f2 = open('test_1.txt', 'a')
for i in range(676044):
    cur_line = customer_purchase_remapid[customer_purchase_remapid['customer'] == i]
    cur_line_new = np.asarray(cur_line['remap_id'])
    if len(cur_line_new) < 5:
        continue
    else:
        train_part_index = np.random.choice(cur_line_new.shape[0], size=int(0.8 * len(cur_line_new)), replace=False)
        train_part = cur_line_new[train_part_index]
        str_cur_train = " ".join([str(x) for x in train_part])
        str_cur_train = str(i) + " " + str_cur_train
        f1.write(str_cur_train)
        f1.write("\n")
        test_part_index = np.delete(np.arange(cur_line_new.shape[0]), train_part_index)
        test_part = cur_line_new[test_part_index]
        str_cur_test = " ".join([str(x) for x in test_part])
        str_cur_test = str(i) + " " + str_cur_test
        f2.write(str_cur_test)
        f2.write("\n")
        
f1.close()
f2.close()